In [41]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import random
from collections import defaultdict
from sklearn import metrics
from time import time
import numpy as np
import re
from nltk.corpus import stopwords
import unicodedata
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

In [2]:
trecho_df = pd.read_csv("2022_Trecho.csv", sep=";", encoding="latin-1")

trecho_df.columns = [
    "id_viagem",
    "numero_proposta",
    "sequencia_trecho",
    "data_origem",
    "pais_origem",
    "uf_origem",
    "cidade_origem",
    "data_destino",
    "pais_destino",
    "uf_destino",
    "cidade_destino",
    "meio_transporte",
    "numero_diarias",
    "missao",
]

In [3]:
viagem_df = pd.read_csv("2022_Viagem.csv", sep=";", encoding="latin-1")

# viagem.csv head:
# "Identificador do processo de viagem";"N�mero da Proposta (PCDP)";"Situa��o";"Viagem Urgente";"Justificativa Urg�ncia Viagem";"C�digo do �rg�o superior";"Nome do �rg�o superior";"C�digo �rg�o solicitante";"Nome �rg�o solicitante";"CPF viajante";"Nome";"Cargo";"Fun��o";"Descri��o Fun��o";"Per�odo - Data de in�cio";"Per�odo - Data de fim";"Destinos";"Motivo";"Valor di�rias";"Valor passagens";"Valor devolu��o";"Valor outros gastos"

viagem_df.columns = [
    "id_viagem",
    "numero_proposta",
    "situacao",
    "viagem_urgente",
    "justificativa_urgencia_viagem",
    "codigo_orgao_superior",
    "nome_orgao_superior",
    "codigo_orgao_solicitante",
    "nome_orgao_solicitante",
    "cpf_viajante",
    "nome",
    "cargo",
    "funcao",
    "descricao_funcao",
    "data_inicio",
    "data_fim",
    "destinos",
    "motivo",
    "valor_diarias",
    "valor_passagens",
    "valor_devolucao",
    "valor_outros_gastos",
]

In [4]:
# calculate the duration of the days of the trip in integer
trecho_df["data_origem"] = pd.to_datetime(trecho_df["data_origem"], format="%d/%m/%Y")
trecho_df["data_destino"] = pd.to_datetime(trecho_df["data_destino"], format="%d/%m/%Y")
trecho_df["duracao"] = (
    trecho_df["data_destino"] - trecho_df["data_origem"]
).dt.days.astype(int)

In [5]:
# convert diarias to float
trecho_df["numero_diarias"] = trecho_df["numero_diarias"].str.replace(",", ".")
trecho_df["numero_diarias"] = trecho_df["numero_diarias"].astype(float)

In [6]:
# convert valores to float
viagem_df["valor_diarias"] = viagem_df["valor_diarias"].str.replace(",", ".")
viagem_df["valor_diarias"] = viagem_df["valor_diarias"].astype(float)
viagem_df["valor_passagens"] = viagem_df["valor_passagens"].str.replace(",", ".")
viagem_df["valor_passagens"] = viagem_df["valor_passagens"].astype(float)
viagem_df["valor_devolucao"] = viagem_df["valor_devolucao"].str.replace(",", ".")
viagem_df["valor_devolucao"] = viagem_df["valor_devolucao"].astype(float)
viagem_df["valor_outros_gastos"] = viagem_df["valor_outros_gastos"].str.replace(
    ",", "."
)
viagem_df["valor_outros_gastos"] = viagem_df["valor_outros_gastos"].astype(float)


In [7]:
list_justificativa = viagem_df[viagem_df["viagem_urgente"] == "SIM"]["justificativa_urgencia_viagem"].tolist()
list_justificativa = [str(x) for x in list_justificativa]
list_justificativa = [x for x in list_justificativa if x != 'nan']
random.seed(42)
list_justificativa = random.sample(list_justificativa, 80000)
len(list_justificativa)

80000

In [19]:
def remove_accents(text):
    return unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('ASCII')

In [22]:
# preprocess text 

# change numbers to X
list_justificativa = [remove_accents(re.sub('\d', 'X', x)).lower() for x in list_justificativa]
print(list_justificativa[10011])


autorizacao atraves da nota tecnica no xxx/xx


In [82]:
# list of stop words
stop_words = stopwords.words("portuguese")
additional_stop = ['conforme', 'devido', 'acordo']
stop_words.extend(additional_stop)
print(stop_words)

['a', 'à', 'ao', 'aos', 'aquela', 'aquelas', 'aquele', 'aqueles', 'aquilo', 'as', 'às', 'até', 'com', 'como', 'da', 'das', 'de', 'dela', 'delas', 'dele', 'deles', 'depois', 'do', 'dos', 'e', 'é', 'ela', 'elas', 'ele', 'eles', 'em', 'entre', 'era', 'eram', 'éramos', 'essa', 'essas', 'esse', 'esses', 'esta', 'está', 'estamos', 'estão', 'estar', 'estas', 'estava', 'estavam', 'estávamos', 'este', 'esteja', 'estejam', 'estejamos', 'estes', 'esteve', 'estive', 'estivemos', 'estiver', 'estivera', 'estiveram', 'estivéramos', 'estiverem', 'estivermos', 'estivesse', 'estivessem', 'estivéssemos', 'estou', 'eu', 'foi', 'fomos', 'for', 'fora', 'foram', 'fôramos', 'forem', 'formos', 'fosse', 'fossem', 'fôssemos', 'fui', 'há', 'haja', 'hajam', 'hajamos', 'hão', 'havemos', 'haver', 'hei', 'houve', 'houvemos', 'houver', 'houvera', 'houverá', 'houveram', 'houvéramos', 'houverão', 'houverei', 'houverem', 'houveremos', 'houveria', 'houveriam', 'houveríamos', 'houvermos', 'houvesse', 'houvessem', 'houvésse

In [83]:
count_vectorizer = CountVectorizer(max_df=0.95, min_df=300, stop_words=stop_words, ngram_range=(1, 3))
X_count = count_vectorizer.fit_transform(list_justificativa)


In [84]:
# count the number of entries > 0
count = (X_count[X_count > 0]).shape[1]
true_k = X_count.shape[0] * X_count.shape[1] // count
# true_k = 100
# cite https://dl.acm.org/doi/pdf/10.1145/99935.99938
print(true_k)

82


In [85]:
sample = list_justificativa
vectorizer = TfidfVectorizer(max_df=0.95, min_df=300, stop_words=stop_words, ngram_range=(1, 3))
X = vectorizer.fit_transform(sample)

# lsa = make_pipeline(TruncatedSVD(n_components=300), Normalizer(copy=False))
# X_lsa = lsa.fit_transform(X)
# explained_variance = lsa[0].explained_variance_ratio_.sum()

# print(f"Explained variance of the SVD step: {explained_variance:.2f}")

model = KMeans(n_clusters=true_k, init='k-means++', max_iter=200, n_init=10, random_state=42).fit(X)


labels=model.labels_
clusters=pd.DataFrame(list(zip(sample,labels)),columns=['phrase','cluster'])
clusters['cluster'].value_counts()

cluster
8     15219
35     3245
71     3237
45     2787
21     2638
      ...  
77      225
67      128
25      128
57      120
4        30
Name: count, Length: 82, dtype: int64

In [86]:
# quantize the clusters
clusters_count = clusters.groupby('cluster').count()
clusters_count = clusters_count.sort_values(by=['phrase'], ascending=False)

clusters_count.head(10)

,phrase
cluster,
8,15219
35,3245
71,3237
45,2787
21,2638
53,1948
51,1883
7,1848
3,1722


In [87]:

#print(clusters.sort_values(by=['cluster']))
# find the centroids
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names_out()

# for i in range(10):
#     print("#Cluster TOP %d" % (i+1)),
#     print("Cluster %d:" % clusters_count.index[i])

#     for ind in order_centroids[clusters_count.index[i], :10]:
#         print(" %s" % terms[ind])

for i in range(true_k):
    print("Cluster %d:" % i),
    print("Cluster size: %d" % clusters_count[clusters_count.index == i].values[0][0])
    for ind in order_centroids[i, :10]:
        print(" %s" % terms[ind])
    print("------------------------------------------")

    for phr in clusters[clusters['cluster']==i]['phrase'].unique()[:20]:
        print("  %s" % phr)
    print("\n")


Cluster 0:
Cluster size: 764
 definido coordenacao viagem
 definido coordenacao
 periodo definido
 periodo definido coordenacao
 definido
 coordenacao viagem
 coordenacao
 periodo
 viagem
 gabinete pessoal
------------------------------------------
  periodo definido pela coordenacao de viagem
  periodo definido pela coordenacao de viagem juntamente com o gabinete pessoal do pr
  data e periodo definido pela coordenacao de viagem juntamente com o gabinete pessoal da pr
  periodo definido pela coordenacao da viagem juntamente com o gabinete pessoal.
  periodo definido pela coordenacao da viagem e o gabinete pessoal.
  periodo definido pelo professor da disciplina.
  periodo definido pela coordenacao de viagem e o gabinete pessoal do pr
  periodo e data definidos pela coordenacao de viagem juntamente com o gabinete pessoal do pr
  periodo definido pela coordenacao de viagem.
  periodo e data definido pela coordenacao de viagem juntamente com o gabinete pessoal do pr
  periodo definido pe

In [88]:
# # print the head 10 of the clusters
    

# print the head 10 of the top 10 clusters
# for i in range(10):
#     print("#Cluster TOP %d" % (i+1)),
#     print("Cluster %d:" % clusters_count.index[i])
    
#     for phr in clusters[clusters['cluster']==clusters_count.index[i]]['phrase'].unique()[:20]:
#         print("  %s" % phr)
#     print("\n")